## This file is used to find common districts between covid dataset and cowin dataset

#### Importing libraries

In [22]:
import pandas as pd
import numpy as np
import json
from fuzzywuzzy import process
from collections import OrderedDict
import bisect 

#### Importing dataset

In [23]:
df = pd.read_csv("cowin_vaccine_data_districtwise.csv")
df.head()

C:\Users\Deepak Raj\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,

,S No,State_Code,State,District_Key,Cowin Key,District,16/01/2021,16/01/2021.1,16/01/2021.2,16/01/2021.3,...,31/10/2021,31/10/2021.1,31/10/2021.2,31/10/2021.3,31/10/2021.4,31/10/2021.5,31/10/2021.6,31/10/2021.7,31/10/2021.8,31/10/2021.9
0,NaN,NaN,NaN,NaN,NaN,NaN,Total Individuals Registered,Sessions,Sites,First Dose Administered,...,Total Doses Administered,Sessions,Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Covaxin (Doses Administered),CoviShield (Doses Administered)
1,1.0,AN,Andaman and Nicobar Islands,AN_Nicobars,Nicobar,Nicobars,745,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,AN,Andaman and Nicobar Islands,AN_North and Middle Andaman,North and Middle Andaman,North and Middle Andaman,1496,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,AN,Andaman and Nicobar Islands,AN_South Andaman,South Andaman,South Andaman,6028,2,2,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,AP,Andhra Pradesh,AP_Anantapur,Anantapur,Anantapur,20781,28,26,287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df2 = pd.read_csv("districts.csv")
df2.head()

,Date,State,District,Confirmed,Recovered,Deceased,Other,Tested
0,2020-04-26,Andaman and Nicobar Islands,Unknown,33,11,0,0,NaN
1,2020-04-26,Andhra Pradesh,Anantapur,53,14,4,0,NaN
2,2020-04-26,Andhra Pradesh,Chittoor,73,13,0,0,NaN
3,2020-04-26,Andhra Pradesh,East Godavari,39,12,0,0,NaN
4,2020-04-26,Andhra Pradesh,Guntur,214,29,8,0,NaN


#### finding no of unique districts in 'districts.csv' dataset

In [25]:
districts1 = df2['District'].dropna().values
districts1 = set(districts1)
len(districts1)

643

#### finding no of unique districts in 'cowin' dataset

In [26]:
cowin_districts = df['District'].dropna().values
# converting to set to remove duplicates
cowin_districts = set(cowin_districts)
# print(cowin_districts)
print(len(cowin_districts))

724


#### dropping irrelevant rows or columns

In [27]:
districts_to_avoid=['Chengalpattu', 'Gaurela Pendra Marwahi', 'Nicobars', 'North and Middle Andaman', 'Saraikela-Kharsawan', 'South Andaman', 'Tenkasi', 'Tirupathur', 'Yanam']
for i in districts_to_avoid:
    cowin_districts.remove(i)
print(len(cowin_districts))

715


In [28]:
# intersection of districts from districts.csv and from cowin database
len(cowin_districts.intersection(districts1))

619

#### importing modified neighbor file

In [29]:
f = open('neighbors_modified.json')
data = json.load(f)
neighbors_districts = {key for key in data}
# print(neighbors_districts)
print(len(neighbors_districts))

705


#### finding common districts and uncommon districts in cowin and ditricts database

In [30]:
matching_dict = {}
unmatched_districts=set()
rem_cowin_districts = cowin_districts.copy()
for district in neighbors_districts:
    if district in cowin_districts:
        matching_dict[district]=district
        rem_cowin_districts.remove(district)
    else:
        unmatched_districts.add(district)
# print(matching_dict)
print("total matched districts are",len(matching_dict))
print('toal unmatched districts are',len(unmatched_districts))

total matched districts are 616
toal unmatched districts are 89


#### for every unmatched district in cowin dataset, finding its closest district in 'district' dataset using fuzzywuzzy

In [31]:
similar=[]
for district in unmatched_districts:
    temp = process.extractOne(district,rem_cowin_districts)
    similar.append([district,temp[0]])
print(len(similar))
similar

89


[['Sant Ravidas Nagar', 'S.A.S. Nagar'],
 ['Tiruvanamalai', 'Tiruvannamalai'],
 ['Jhunjhunun', 'Jhunjhunu'],
 ['Dhaulpur', 'Dholpur'],
 ['Belgaum', 'Belagavi'],
 ['Ri-Bhoi', 'Ribhoi'],
 ['Sivagangai', 'Sivaganga'],
 ['Maldah', 'Malda'],
 ['Firozpur', 'Ferozepur'],
 ['Seraikela Kharsawan', 'Maharajganj'],
 ['Sri Potti Sriramulu Nellore', 'S.P.S. Nellore'],
 ['Kochbihar', 'Cooch Behar'],
 ['Bangalore Rural', 'Bengaluru Rural'],
 ['Sharawasti', 'Shrawasti'],
 ['Sahibzada Ajit Singh Nagar', 'S.A.S. Nagar'],
 ['Puruliya', 'Purulia'],
 ['Sri Ganganagar', 'Ganganagar'],
 ['North Parganas', 'North 24 Parganas'],
 ['Mahesana', 'Mehsana'],
 ['Kaimur (Bhabua)', 'Kaimur'],
 ['Sabar Kantha', 'Sabarkantha'],
 ['Jagatsinghapur', 'Jagatsinghpur'],
 ['South Salmara-Mankachar', 'South Salmara Mankachar'],
 ['Shopian', 'Shopiyan'],
 ['Chamarajanagar', 'Chamarajanagara'],
 ['Jajapur', 'Jajpur'],
 ['Devbhumi Dwaraka', 'Devbhumi Dwarka'],
 ['Tiruchchirappalli', 'Tiruchirappalli'],
 ['Bellary', 'Ballari'],
 

#### manually found districts for which fuzzywuzzy process gave wrong result

In [32]:
dissimilar_districts = [['Ysr', 'Balasore'],
 ['Jyotiba Phule Nagar', 'S.A.S. Nagar'],
 ['Pashchim Champaran', 'East Champaran'],
 ['Mumbai Suburban', 'Mumbai'],
 ['Sri Potti Sriramulu Nellore', 'Sri Muktsar Sahib'],
 ['Purbi Singhbhum', 'West Singhbhum'],
 ['Sant Ravidas Nagar', 'S.A.S. Nagar'],
 ['Seraikela Kharsawan', 'Maharajganj'],
 ['Baleshwar', 'Ballari'],
 ['Hugli', 'Lahaul and Spiti'],
 ['Palghat', 'Parbhani'],
 ['Mumbai City', 'Mumbai'],
 ['Bid', 'Karbi Anglong'],]

#### removing all wrong predicted districts from 'similar' set

In [33]:
for district in dissimilar_districts:
    if district in similar:
        similar.remove(district)
    else:
        print(district)
print("found correct district names using fuzzywuzzy of total "+str(len(similar))+" districts")

['Ysr', 'Balasore']
['Sri Potti Sriramulu Nellore', 'Sri Muktsar Sahib']
['Purbi Singhbhum', 'West Singhbhum']
['Palghat', 'Parbhani']
found correct district names using fuzzywuzzy of total 80 districts


#### manually corrected some districts name

In [34]:
# not found mumbai suburban, mumbai city and narsimhapur
correct_districts = [
['Seraikela Kharsawan', 'Saraikela-Kharsawan'],
['Pashchim Champaran', 'West Champaran'],
['Hugli', 'Hooghly'],
['Sri Potti Sriramulu Nellore', 'S.P.S. Nellore'],
['Purbi Singhbhum', 'East Singhbhum'],
['Palghat', 'Palakkad'],
['Ysr', 'Y.S.R. Kadapa'],
['Baleshwar', 'Balasore'],
['Bid', 'Beed'],
['Sant Ravidas Nagar', 'Bhadohi'],
['Jyotiba Phule Nagar', 'Amroha']]

In [35]:
for district in correct_districts:
    if district not in similar:
        similar.append(district)
print(len(similar))
# similar

88


In [36]:
# finally adding similar districts to our matching district dictionary
for item in similar:
    matching_dict[item[0]]=item[1]
# print(matching_dict)
print(len(matching_dict))
print("out of "+str(len(neighbors_districts))+" districts in neigbors_district, total "+str(len(matching_dict))+" districts matching found in cowin database")

703
out of 705 districts in neigbors_district, total 703 districts matching found in cowin database


In [37]:
for key in matching_dict:
    temp = df.loc[df['District'] == matching_dict[key], 'District_Key']
    if(len(temp)==0):
        print("district not found in cowin database",matching_dict[key])
    else:
        matching_dict[key]=[matching_dict[key], temp.iloc[0]]
matching_dict

{'Palwal': ['Palwal', 'HR_Palwal'],
 'Saharsa': ['Saharsa', 'BR_Saharsa'],
 'Golaghat': ['Golaghat', 'AS_Golaghat'],
 'Upper Subansiri': ['Upper Subansiri', 'AR_Upper Subansiri'],
 'South West Delhi': ['South West Delhi', 'DL_South West Delhi'],
 'Kupwara': ['Kupwara', 'JK_Kupwara'],
 'Koppal': ['Koppal', 'KA_Koppal'],
 'Kurung Kumey': ['Kurung Kumey', 'AR_Kurung Kumey'],
 'Mansa': ['Mansa', 'PB_Mansa'],
 'Nagpur': ['Nagpur', 'MH_Nagpur'],
 'Akola': ['Akola', 'MH_Akola'],
 'Gopalganj': ['Gopalganj', 'BR_Gopalganj'],
 'Rohtak': ['Rohtak', 'HR_Rohtak'],
 'Kamrup': ['Kamrup', 'AS_Kamrup'],
 'Kendrapara': ['Kendrapara', 'OR_Kendrapara'],
 'Bhandara': ['Bhandara', 'MH_Bhandara'],
 'Uttar Dinajpur': ['Uttar Dinajpur', 'WB_Uttar Dinajpur'],
 'Udham Singh Nagar': ['Udham Singh Nagar', 'UT_Udham Singh Nagar'],
 'Hanumangarh': ['Hanumangarh', 'RJ_Hanumangarh'],
 'North East Delhi': ['North East Delhi', 'DL_North East Delhi'],
 'Theni': ['Theni', 'TN_Theni'],
 'East Godavari': ['East Godavari', '

#### getting all neighbors of matching districts

In [38]:
f = open("neighbors_modified.json")
districts_json = json.load(f)
final_dict = {}
for district in districts_json:
    if district in matching_dict:
        key = matching_dict[district][1]
        values = []
        for neighbor in districts_json[district]:
            if neighbor in matching_dict:
                bisect.insort(values,matching_dict[neighbor][1])
            else:
                print("district not found",neighbor)
        final_dict[key]=values
    else:
        print("district not found",district)
final_dict

district not found Mumbai Suburban
district not found Mumbai Suburban
district not found Mumbai City


{'HP_Lahaul and Spiti': ['HP_Chamba',
  'HP_Kangra',
  'HP_Kinnaur',
  'HP_Kullu',
  'JK_Kishtwar',
  'LA_Kargil',
  'LA_Leh'],
 'HP_Kinnaur': ['HP_Kullu',
  'HP_Lahaul and Spiti',
  'HP_Shimla',
  'UT_Uttarkashi'],
 'HP_Kullu': ['HP_Kangra',
  'HP_Kinnaur',
  'HP_Lahaul and Spiti',
  'HP_Mandi',
  'HP_Shimla'],
 'HP_Chamba': ['HP_Kangra',
  'HP_Lahaul and Spiti',
  'JK_Doda',
  'JK_Kathua',
  'JK_Kishtwar',
  'PB_Pathankot'],
 'HP_Kangra': ['HP_Chamba',
  'HP_Kullu',
  'HP_Lahaul and Spiti',
  'HP_Mandi',
  'HP_Una',
  'PB_Gurdaspur',
  'PB_Hoshiarpur',
  'PB_Pathankot'],
 'LA_Leh': ['HP_Lahaul and Spiti', 'LA_Kargil'],
 'LA_Kargil': ['HP_Lahaul and Spiti',
  'JK_Anantnag',
  'JK_Bandipora',
  'JK_Ganderbal',
  'JK_Kishtwar',
  'LA_Leh'],
 'JK_Kishtwar': ['HP_Chamba',
  'HP_Lahaul and Spiti',
  'JK_Anantnag',
  'JK_Doda',
  'LA_Kargil'],
 'HP_Shimla': ['HP_Kinnaur',
  'HP_Kullu',
  'HP_Mandi',
  'HP_Sirmaur',
  'HP_Solan',
  'UT_Dehradun',
  'UT_Uttarkashi'],
 'UT_Uttarkashi': ['HP_Ki

#### adding some more districts which were corrected manually

In [39]:
f1 = open("duplicate_districts.json")
duplicate_districts = json.load(f1)
for district in duplicate_districts:
    final_dict[district] = duplicate_districts[district]
for district in duplicate_districts:
    for value in duplicate_districts[district]:
        if district not in final_dict[value]:
            bisect.insort(final_dict[value],district)

In [40]:
print("Total districts in final neighbor district file are",len(final_dict))

Total districts in final neighbor district file are 713


In [42]:
final_dict = OrderedDict(sorted(final_dict.items()))
with open("cowin_district_intersection.json", "w") as file:
    json.dump(final_dict, file, indent=4)